The dataset we’ll be working with is for e-commerce from a real UK retailer, so the data is 100% real. The problems we encounter here will probably show up in other business data as well.

In [ ]:
There are 8 columns:
InvoiceNo — an integer representing the invoice number. Example: 536365.
StockCode — a unique string of alphanumeric characters representing the purchased item. Example: 85123A.
Description — a description of the product. Example: WHITE HANGING HEART T-LIGHT HOLDER.
Quantity — the number of items purchased. Example: 6.
InvoiceDate — the invoice date. Example: 12/1/2010 8:26.
UnitPrice — the price of one of the products. Example: 2.55.
CustomerID — a unique integer representing a customer. Example: 17850.
Country — the country of the customer. Example: United Kingdom.
This is a classic example of customer segmentation data.

Our job is to achieve customer-centric data, where each new row represents a unique customer whose attributes are listed in other columns.
There are some other challenges, too; the description is the only indicator of a product. One solution to integrate what products the customer buys into user-centric data would be to make each unique product a column, with the value in a column representing how many of each product a user bought. Unfortunately, this is infeasible both by the extremely high number of dimensions, something that Euclidean-distance based algorithms like K-Means shouldn’t operate in due to the Curse of Dimensionality, and because it’s completely un-interpretable even if the analysis is complete, not to mention how difficult a Principal Component Analysis for dimensionality reduction would be.
To address this problem, we’ll cluster product descriptions into a few buckets (at most, 4 or 5). Each of these product clusters will be its own column, with its cell values indicating the total value of the items a customer bought that fit into that bucket. This lowers the number of dimensions from our previous solution and is more interpretable.

In [ ]:
Let’s set up a process.
1.Load the data and do basic cleaning (deleting identical rows, etc.).
2.Vectorize the descriptions of all the items.
3.Cluster them into a maximum of 5 clusters, and create columns from them based on how much money customers spent in each item category.
4.We want to represent a customer’s buying record in a few columns. To do this, create five columns: count, minimum, maximum, mean, and sum, which each represent some statistical aspect of a list with all the total prices of orders a customer has made.
5.Create a binary column that represents if the customer is foreign or not (from the UK or not). There are 38 unique countries listed in the dataset, and each cannot be its own column. Many companies simply list customers as foreign or not, and since this ecommerce company is based in the UK, all customers outside of the UK are considered foreign.
6.Cluster the customers using K-Means, and take analyses from the cluster centers.
7.Use the K-Means model to make analyses.

# Step 1: Loading and Cleaning the Data

In [9]:
import pandas as pd
data = pd.read_csv(r'C:\Users\Smita Gavandi\Documents\python_excelR\LENOVO\R Excel Sessions\Assignments\endtoend_projects\segmentation_ecommerce\data.csv',encoding='ISO-8859-1')
data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


In [11]:
data.columns

Index(['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'UnitPrice', 'CustomerID', 'Country'],
      dtype='object')

In [3]:
len(data)

541909

In [5]:
len(data['CustomerID'].unique())

4373

In [12]:
data['CustomerID'].isnull().sum()

135080

In [14]:
#number of rows that would be dropped if we deleted any row that had a na value.
len(data) - len(data.dropna())

135080

For the purposes of this project, I’ll be dropping all rows that have na values. 

In [15]:
data = data.dropna()
data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


In [16]:
len(data)

406829

The InvoiceNo and StockCode columns are irrelevant. We can remove them

In [17]:
data.drop(['InvoiceNo','StockCode'],axis=1,inplace=True)

In [18]:
data.head()

,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


Next, we’ll be vectorizing the Description column to cluster products into groups. To prepare the data for this, we’ll need to make the words lowercase and remove any punctuation, for example in row 4 with “RED WOOLY HOTTIE WHITE HEART.”

I'll make a function that cleans the description. It uses regular expressions to remove punctuation and lowers capital letters.

In [19]:
import re
def clean(description):
    return re.sub(r'[^\w\s]','',description).lower()

In [20]:
data['Description']=data['Description'].apply(clean)

In [21]:
data.head()

,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,white hanging heart tlight holder,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,white metal lantern,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,cream cupid hearts coat hanger,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,knitted union flag hot water bottle,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,red woolly hottie white heart,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


we need to reset the index and quickly create a new column. Because we removed all na values, there are going to be some holes in the indexes. If let untamed, it will cause problems for us in the future.

In [23]:
data = data.reset_index().drop('index',axis=1)
data.head()

,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,white hanging heart tlight holder,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,white metal lantern,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,cream cupid hearts coat hanger,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,knitted union flag hot water bottle,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,red woolly hottie white heart,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


In the future, we will need to know the total amount of money spent on a product — rephrased, the unit cost times the quantity of items.

In [24]:
data['Total'] =data['UnitPrice']*data['Quantity']

# Step 2: Vectorize the Description

Given the nature of these descriptions, it’s probably best to use bag of words instead of TD-IDF, which is more suited for long texts that have more redundant words. It seems that every word in the description is a keyword of sorts, so vectorizing it simply with bag of words/count is the best way to go.

In [25]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(stop_words='english')

This creates a CountVectorizer object with English stop words (words like and, of, a, etc. that don’t convey much meaning). We can then fit the vectorizer on the data and transform it using .fit_transform.

In [26]:
descriptions = vectorizer.fit_transform(data['Description'])
descriptions

<406829x2093 sparse matrix of type '<class 'numpy.int64'>'
	with 1640936 stored elements in Compressed Sparse Row format>

Now, we can cluster the products.

# Step 3: Cluster the Products & Create Features

In [27]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=5)
kmeans.fit(descriptions)

KMeans(n_clusters=5)

We can assign each item purchased whatever label it got through the KMeans into a new column, Product, with this code:

In [28]:
data['Product'] = kmeans.labels_
data.head()

,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Total,Product
0,white hanging heart tlight holder,6,12/1/2010 8:26,2.55,17850.0,United Kingdom,15.30,2
1,white metal lantern,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,20.34,1
2,cream cupid hearts coat hanger,8,12/1/2010 8:26,2.75,17850.0,United Kingdom,22.00,1
3,knitted union flag hot water bottle,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,20.34,1
4,red woolly hottie white heart,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,20.34,2


In [29]:
customers = pd.DataFrame({'CustomerID':data['CustomerID'].unique()})
customers.head()

,CustomerID
0,17850.0
1,13047.0
2,12583.0
3,13748.0
4,15100.0


In [30]:
len(customers)

4372

Next, we’re going to need to create 5 columns, one for each product category. For now, we’ll set it equal to 0.

In [31]:
customers['Product0'] = 0
customers['Product1'] = 0
customers['Product2'] = 0
customers['Product3'] = 0
customers['Product4'] = 0
customers.head()

,CustomerID,Product0,Product1,Product2,Product3,Product4
0,17850.0,0,0,0,0,0
1,13047.0,0,0,0,0,0
2,12583.0,0,0,0,0,0
3,13748.0,0,0,0,0,0
4,15100.0,0,0,0,0,0


From here, we’ll use the fill-in method. 
For each row in data, we’ll add whatever amount of money spent on that product to whatever product category it is in to the row that has a matching CustomerID.

In [33]:
from tqdm import tqdm
for index in tqdm(range(len(data))):
    customerID = data.loc[index,'CustomerID']
    product_category = data.loc[index,'Product']
    product_price = data.loc[index,'Total']
    customers_table_index = customers[customers['CustomerID']==customerID].index[0]
    customers.loc[customers_table_index, 'Product'+str(product_category)] += product_price

100%|█████████████████████████████████████████████████████████████████████████| 406829/406829 [08:03<00:00, 841.97it/s]


In [34]:
customers.head()

,CustomerID,Product0,Product1,Product2,Product3,Product4
0,17850.0,321.30,3803.27,668.46,0.0,495.60
1,13047.0,518.38,2288.91,161.06,0.0,110.75
2,12583.0,1373.34,4406.56,0.00,177.6,1229.84
3,13748.0,52.08,463.59,34.30,0.0,398.28
4,15100.0,0.00,635.10,0.00,0.0,0.00


# Step 4 | Creating Purchase History Descriptions

In [ ]:
Next, our job is to create five columns: count, maximum, minimum, average, and sum, which are five numerical indicators of a customer’s purchase history. 

Note that we need to do this because the alternative, which is to encode every customer’s purchase history by date, 
would take up several hundred columns. Like discussed before, this is infeasible.

We will use a similar method as before. Initially, we will set the 5 columns to 0 and fill them in iteratively.

In [35]:
customers['PurchaseCount'] = 0
customers['PurchaseMin'] = 0
customers['PurchaseMax'] = 0
customers['PurchaseMean'] = 0
customers['PurchaseSum'] = 0
customers.head()

,CustomerID,Product0,Product1,Product2,Product3,Product4,PurchaseCount,PurchaseMin,PurchaseMax,PurchaseMean,PurchaseSum
0,17850.0,321.30,3803.27,668.46,0.0,495.60,0,0,0,0,0
1,13047.0,518.38,2288.91,161.06,0.0,110.75,0,0,0,0,0
2,12583.0,1373.34,4406.56,0.00,177.6,1229.84,0,0,0,0,0
3,13748.0,52.08,463.59,34.30,0.0,398.28,0,0,0,0,0
4,15100.0,0.00,635.10,0.00,0.0,0.00,0,0,0,0,0
